In [15]:
import numpy as np
import pandas as pd

In [16]:
df = pd.read_csv("data.csv", parse_dates=['Date and Time of initial call'])
df

/tmp/ipykernel_67660/1112248088.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv("data.csv", parse_dates=['Date and Time of initial call'])


,Date and Time of initial call,Date and time of Ranger response,Borough,Property,Location,Species Description,Call Source,Species Status,Animal Condition,Duration of Response,...,311SR Number,Final Ranger Action,# of Animals,PEP Response,Animal Monitored,Rehabilitator,Hours spent monitoring,Police Response,ESU Response,ACC Intake Number
0,2021-06-23 16:45:00,06/24/2021 08:00:00 AM,Brooklyn,Sternberg Park,Inside locked athletic field under construction,Chukar,Other,Exotic,Healthy,6.00,...,311-06712416,ACC,6.0,False,False,NaN,NaN,False,False,163537
1,2021-06-24 10:00:00,06/24/2021 11:00:00 AM,Bronx,Haffen Park,Haffen Pool,Sparrow,Central,Native,Healthy,1.75,...,311-06714879,Rehabilitator,4.0,False,False,NaN,NaN,False,False,NaN
2,2021-06-23 14:30:00,06/23/2021 02:30:00 PM,Bronx,Pelham Bay Park,Pelham Bay South,White-tailed Deer,Employee,Native,NaN,1.00,...,NaN,Unfounded,0.0,False,False,NaN,NaN,False,False,NaN
3,2021-06-23 13:00:00,06/23/2021 01:10:00 PM,Staten Island,Willowbrook Park,The carousel,Raccoon,Employee,Native,NaN,2.00,...,NaN,Unfounded,0.0,False,False,NaN,NaN,False,False,NaN
4,2021-06-23 09:20:00,06/23/2021 09:20:00 AM,Queens,Judge Moses Weinstein Playground,Garbage can,Virginia Opossum,Central,Native,Healthy,2.25,...,311-06699415,ACC,1.0,False,False,NaN,NaN,False,False,119833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2604,2018-06-05 00:00:00,06/05/2018 12:01:00 AM,Manhattan,Abingdon Square,NaN,raccoon,Central,Native,Healthy,0.75,...,NaN,Relocated/Condition Corrected,1.0,False,True,NaN,1.0,False,False,NaN
2605,2018-06-01 12:00:00,06/01/2018 12:30:00 PM,Manhattan,Central Park,park,Raccoon,Employee,Native,Injured,1.25,...,1-1-1568786600,ACC,1.0,False,False,NaN,NaN,False,False,36061
2606,2018-05-16 09:00:00,05/17/2018 10:10:00 AM,Manhattan,Morningside Park,NaN,Raccoon,Employee,Native,DOA,1.50,...,NaN,ACC,2.0,False,True,NaN,0.5,False,False,28316
2607,2018-05-02 09:30:00,05/02/2018 12:00:00 PM,Manhattan,Central Park,park sewer drain,Raccoon,Public,Native,Healthy,0.75,...,NaN,Unfounded,0.0,NaN,NaN,NaN,NaN,NaN,False,NaN


In [17]:
importable_table = df[['Date and Time of initial call', 
                    'Species Description',
                    'Call Source',
                    'Final Ranger Action',
                    'Animal Class',
                    '# of Animals']].copy()

In [18]:
# Extract year from datetime
importable_table['Date and Time of initial call'] = pd.to_datetime(importable_table['Date and Time of initial call'])
importable_table['Year'] = importable_table['Date and Time of initial call'].dt.year

In [19]:
importable_table['Year']

0       2021
1       2021
2       2021
3       2021
4       2021
        ... 
2604    2018
2605    2018
2606    2018
2607    2018
2608    2018
Name: Year, Length: 2609, dtype: int32

# 1. Animals sent to rehabilitators/care centers over time

In [20]:
rehab_analysis = (
    importable_table[importable_table['Final Ranger Action'].str.contains('rehabilitator|care center', case=False, na=False)]
    .groupby('Year')['# of Animals']
    .sum()
    .reset_index()
    .rename(columns={'# of Animals': 'Animals Sent to Rehabilitation'})
)

rehab_analysis

,Year,Animals Sent to Rehabilitation
0,2018,45.0
1,2019,78.0
2,2020,151.0
3,2021,75.0


# 2. Animal types over time

In [21]:

animal_type_analysis = (
    importable_table.groupby(['Year', 'Animal Class'])['# of Animals']
    .sum()
    .reset_index()
    .sort_values(['Year', '# of Animals'], ascending=[True, False])
    .groupby('Year')
    .first()
    .reset_index()
    .rename(columns={'Animal Class': 'Most Common Animal Type'})
)

animal_type_analysis

,Year,Most Common Animal Type,# of Animals
0,2018,Small Mammals-RVS,400.0
1,2019,Domestic,257.0
2,2020,Fish-numerous quantity,901.0
3,2021,Birds,182.0


# 3. Call source changes over time

In [22]:
call_source_analysis = (
    importable_table.groupby(['Year', 'Call Source'])['# of Animals']
    .count()
    .reset_index()
    .sort_values(['Year', '# of Animals'], ascending=[True, False])
    .groupby('Year')
    .first()
    .reset_index()
    .rename(columns={'Call Source': 'Most Common Call Source'})
)

call_source_analysis

,Year,Most Common Call Source,# of Animals
0,2018,"Conservancies/""Friends of"" Groups",258
1,2019,Employee,213
2,2020,Employee,281
3,2021,Employee,147


# Create publication table

In [23]:
publication_table = pd.merge(rehab_analysis, animal_type_analysis, on='Year')
publication_table = pd.merge(publication_table, call_source_analysis[['Year', 'Most Common Call Source']], on='Year')

with pd.ExcelWriter('Urban_Rangers_Analysis.xlsx', engine='openpyxl') as writer:
    importable_table.to_excel(writer, sheet_name='Importable Table', index=False)
    publication_table.to_excel(writer, sheet_name='Publication Table', index=False, startrow=2)
    workbook = writer.book
    worksheet = writer.sheets['Publication Table']
    caption = """Table 1: NYC Urban Park Rangers Animal Assistance Analysis (2018-2024)
    
    This table presents annual trends in animal assistance cases, showing:
    1. Total number of animals sent to rehabilitation centers
    2. Most frequently reported animal type
    3. Most common source of initial reports
    
    Methodology:
    - Data filtered for rehabilitation/care center cases
    - Animal counts aggregated annually
    - Most common animal types and call sources determined by frequency
    """
    worksheet.cell(row=1, column=1, value=caption)